# Tips on Error propagation

## Error propagation reminder

Suppose we have a function $y = f(x_1, x_2, ..., x_n)$. Then we can propagate uncertainties $\sigma_{x_i}$ using

$\sigma_{y}^2 = \sum_i (\frac{\partial f}{\partial x_i} \sigma_{x_i})^2$

E.g. if we have $y = f(a, b) = a + b$ we get for $a=10\pm2$ and $b=20\pm3$

In [ ]:
import math

In [ ]:
a = 10
b = 20
sigma_a = 2
sigma_b = 3

y = a + b
dyda = 1
dydb = 1
sigma_y = math.sqrt((dyda * sigma_a) ** 2 + (dydb * sigma_b) ** 2)

f"y = {y}+/-{sigma_y}"

## Correlations

But what if we have correlated values, e.g. $y = f(c, d) = c + d$ with $c = a + b$ and $d = a \cdot b$?

Now $c$ and $d$ are correlated (because they both depend on $a$ and $b$).

One way is of course to write $f$ as a function of the uncorrelated variables $a$ and $b$, $f(a, b) = a + b + a \cdot b$ and get

$\sigma_y^2 = (\frac{\partial f}{\partial a}\sigma_a)^2 + (\frac{\partial f}{\partial b}\sigma_b)^2 = ((1 + b)\sigma_a)^2 + ((1 + a)\sigma_b)^2$

In [ ]:
y = a + b + a * b
sigma_y = math.sqrt(((1 + b) * sigma_a) ** 2 + ((1 + a) * sigma_b) ** 2)
f"y = {y}+/-{sigma_y}"

Alternatively we use the more general error propagation function for the case where values can be correlated (see https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Non-linear_combinations)

$\Sigma^f \approx J\Sigma^xJ^T$

or in components

$\Sigma_{kl}^f = \mathrm{cov}(f_k, f_l) \approx \sum_{ij} \frac{\partial f_k}{\partial x_i}\frac{\partial f_l}{\partial x_j} \mathrm{cov}(x_i, x_j)$

So in this example we could then first get the covariance of $c = a + b$ and $d = a \cdot b$ via

$
\mathrm{cov}(c, d) \approx \frac{\partial c}{\partial a}\frac{\partial d}{\partial a} \sigma_a^2 + \frac{\partial c}{\partial b}\frac{\partial d}{\partial b} \sigma_b^2
= b \sigma_a^2 + a \sigma_b^2
$

In [ ]:
cov_cd = b * sigma_a**2 + a * sigma_b**2
cov_cd

Then we get the propagated uncertainty as

$\sigma_c^2 = (\frac{\partial c}{\partial a}\sigma_a)^2 + (\frac{\partial c}{\partial b}\sigma_b)^2$

$\sigma_d^2 = (\frac{\partial d}{\partial a}\sigma_a)^2 + (\frac{\partial d}{\partial b}\sigma_b)^2$

$\sigma_y^2 = (\frac{\partial f}{\partial c}\sigma_c)^2 + (\frac{\partial f}{\partial d}\sigma_d)^2 + 2\frac{\partial f}{\partial d}\frac{\partial f}{\partial c}\mathrm{cov}(c, d)$

In [ ]:
sigma_c = math.sqrt(sigma_a**2 + sigma_b**2)
sigma_d = math.sqrt((b * sigma_a) ** 2 + (a * sigma_b) ** 2)
sigma_y = math.sqrt(sigma_c**2 + sigma_d**2 + 2 * cov_cd)

f"y = {y}+/-{sigma_y}"

## Use the uncertainties package

We don't need to calculate all that by hand (annoying especially for more complicated formulae). A very nice package from this is `uncertainties` (https://uncertainties.readthedocs.io)

In [ ]:
#!pip install uncertainties

In [ ]:
import uncertainties
from uncertainties import ufloat

We can define `ufloat` numbers that are numbers with errors:

In [ ]:
a = ufloat(10, 2)
b = ufloat(20, 3)

In [ ]:
a

In [ ]:
b

Now, if we calculate something using these numbers, the uncertainties will be automatically propagated:

In [ ]:
c = a + b
c

In [ ]:
d = a * b
d

In [ ]:
y = c + d
y

This took the correlation of c and d into account. We can also retrieve the covariance matrix by

In [ ]:
uncertainties.covariance_matrix([c, d])

Or the other way round, if we already have the covariance matrix between values (e.g. from a previous calculation or a fit result) we can pass it to uncertainties:

In [ ]:
cc, dd = uncertainties.correlated_values(
    [30, 200],
    [[13.0, 170.0], [170.0, 2500.0]]
)
ff = cc + dd
ff

The `uncertainties.umath` module has the standard mathematical functions, so one can use these for `ufloat` values as well

In [ ]:
from uncertainties import umath

In [ ]:
dir(umath)

In [ ]:
umath.sqrt(ff)

## Using the jacobi package

Sometimes we don't have a closed analytical form for our functions, so we may need to calculate the derivatives numerically. One package to do this is `jacobi` (https://hdembinski.github.io/jacobi):

In [ ]:
#!pip install jacobi

In [ ]:
import jacobi

In [ ]:
def f(args):
    c, d = args
    return c + d

In [ ]:
y, ycov = jacobi.propagate(
    f,
    [30, 200],
    [[13.0, 170.0], [170.0, 2500.0]]
)

f"{y}+/-{math.sqrt(ycov)}"

The advantage is that this will work for any arbitrary function even if it is not composable to operations that can act on `ufloat` values from uncertainties

## Manual numerical propagation

As a final method one can also manually propagate uncertainties through an arbitrary function by varying the inputs with +/- 1 standard deviation. Then in the error propagation formula we can replace

$\frac{\partial f}{\partial x}\sigma_x \approx \frac{f(x + \sigma_x) - f(x - \sigma_x)}{2}$

We then have to rewrite the covariance using the correlation:

$\mathrm{corr}({x_i, x_j}) = \frac{\mathrm{cov}(x_i, x_j)}{\sigma_i\sigma_j}$

So we can write for the mixed terms

$\frac{\partial f}{\partial x_i}\frac{\partial f}{\partial x_j}\mathrm{cov}(x_i, x_j) \approx \frac{f(x_i + \sigma_i, x_j) - f(x_i - \sigma_i, x_j)}{2}\frac{f(x_i, x_j + \sigma_j) - f(x_i, x_j - \sigma_j)}{2}\mathrm{corr}({x_i, x_j})$

With our simple example:

In [ ]:
corrcoef = 170 / math.sqrt(13*2500)
corrcoef

E.g. again for our simple example:

In [ ]:
sigma_c = math.sqrt(13)
sigma_d = math.sqrt(2500)

delta_c = (f((30 + sigma_c, 200)) - f((30 - sigma_c, 200))) / 2
delta_d = (f((30, 200 + sigma_d)) - f((30, 200 - sigma_d))) / 2

In [ ]:
dy = math.sqrt(delta_c**2 + delta_d**2 + 2 * delta_c * delta_d * corrcoef)
dy

Also this would work for arbitrary functions

## With Monte Carlo methods ("toys")

For complicated functions with a huge amount of parameters it might be worth sampling a bunch of values from the uncertain function inputs and check the standard deviation of the resulting function output:

In [ ]:
import numpy as np

In [ ]:
random_inputs = np.random.multivariate_normal(mean=[30, 200], cov=[[13.0, 170.0], [170.0, 2500.0]], size=100)

In [ ]:
random_outputs = [f((c, d)) for c, d in random_inputs]

In [ ]:
np.std(random_outputs)

This will of course fluctuate in the order of $\sqrt{N}$ depending on the size $N$ of the toy sample (here for $N=100$ we excpect a precision in the order of 10%)